In [1]:
# Carregue as bibliotecas necessárias
import json
import pickle
import nltk
import random
import numpy as np
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# Ferramenta de NLP
from textblob import TextBlob
# Ferramenta para visualização dos Dados
from wordcloud import WordCloud, STOPWORDS
# Ferramentas do pacote NLTK
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize , download , pos_tag
# Ferramentas d lib padrão python3
import warnings
import re
import string
# Download de mais Ferramentas da biblioteca do NLTK
download(['punkt','averaged_perceptron_tagger','stopwords','wordnet','omw-1.4'])
stopwords = stopwords.words('portuguese')
# Para esse projeto usaremos a Técnica de Lematização, pois queremos ter a certeza de que a forma reduzida de uma palavra existe (radical)
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
database = {
  "intents": [
    {
      "tag": "welcome",
      "patterns": ["Oi","Oi, bom dia","Oi, boa tarde", "bom dia", "boa tarde", "boa noite", "oi, boa noite", "olá, boa noite", "oiiiii", "Olá","oiii, como vai?","opa, tudo bem?"],
      "responses": ["Olá, serei seu assistente virtual, em que posso te ajudar?","Salve, qual foi ?", "Manda pro pai, Lança a braba", "No que posso te ajudar ?"],
      "context": [""]
    },
    {
      "tag": "my_classes",
      "patterns": ["Quais são as minhas matérias ?","Quais são as minhas matérias de hoje ? ","Quais são as minhas disciplinas de hoje ? ", "Que aulas eu tenho Hoje","me fale minhas turmas", "que sala eu devo ir?", "Qual minha Sala ?","quais as minhas turmas ?"],
      "responses": ["Entendi, você deseja saber suas salas","Você deseja saber suas salas ?", "Ah, você quer saber qual sala ? ", "Suas Aulas ?"],
      "context": ["RA"]
    },
    {
      "tag": "anything_else",
      "patterns": [],
      "responses": ["Desculpa, não entendi o que você falou, tente novamente!","Não compreendi a sua solicitação, talvez eu possa te ajudar"],
      "context": [""]
    }
  ]
}

In [3]:
my_classes_triggers = database['intents'][1]['patterns'] + [ "meu ra é " + sentence for sentence in database['intents'][1]['patterns'] ]

display(my_classes_triggers)
database['intents'][1]['patterns'] = my_classes_triggers

['Quais são as minhas matérias ?',
 'Quais são as minhas matérias de hoje ? ',
 'Quais são as minhas disciplinas de hoje ? ',
 'Que aulas eu tenho Hoje',
 'me fale minhas turmas',
 'que sala eu devo ir?',
 'Qual minha Sala ?',
 'quais as minhas turmas ?',
 'meu ra é Quais são as minhas matérias ?',
 'meu ra é Quais são as minhas matérias de hoje ? ',
 'meu ra é Quais são as minhas disciplinas de hoje ? ',
 'meu ra é Que aulas eu tenho Hoje',
 'meu ra é me fale minhas turmas',
 'meu ra é que sala eu devo ir?',
 'meu ra é Qual minha Sala ?',
 'meu ra é quais as minhas turmas ?']

In [4]:
database['intents'][1]['patterns'] = ['Quais são as minhas matérias ?',  'Quais são as minhas matérias de hoje ? ',  'Quais são as minhas disciplinas de hoje ? ',  'Que aulas eu tenho Hoje',  'me fale minhas turmas',  'que sala eu devo ir?',  'Qual minha Sala ?',  'quais as minhas turmas ?',  'meu ra é Quais são as minhas matérias ?',  'meu ra é Quais são as minhas matérias de hoje ? ',  'meu ra é Quais são as minhas disciplinas de hoje ? ',  'meu ra é Que aulas eu tenho Hoje',  'meu ra é me fale minhas turmas',  'meu ra é que sala eu devo ir?',  'meu ra é Qual minha Sala ?',  'meu ra é quais as minhas turmas ?', 'Quero saber minhas turmas', 'Quero saber minhas salas', 'Quero saber minhas disciplinas', 'Quero saber minhas turmas meu ra é', 'Quero saber minhas salas meu ra é', 'Quero saber minhas disciplinas meu ra é', 'meu ra é Quero saber minhas turmas', 'meu ra é Quero saber minhas salas', 'meu ra é Quero saber minhas disciplinas', 'Que aula hoje ?', 'Que turma hoje ?', 'Que disciplina hoje ?', 'Que aula agora ?', 'Que turma agora ?', 'Que disciplina agora ?', 'Que matéria hoje ?', 'Que máteria agora ?', 'Quais minhas próximas aulas ?', 'Qual minha próxima turma ?', 'Qual minha próxima sala ?', 'Quero saber minha próxima aula', 'Quero saber minha próxima turma', 'Quero saber minha próxima sala']

In [5]:
database['intents'][0]['patterns'] = 7*database['intents'][0]['patterns']

In [6]:

def limpa_emotes(texto):
  """
    Função para Limpar emotes dos Comentários
    
    Exemplo :  🤔 ?
  """
  emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  
        u"\u3030"
        "]+",
                      re.UNICODE)
  return re.sub(emoj, '', texto)

def limpa_urls(texto) : 
  """
  Função para Limpar emotes dos Comentários
    
    Exemplo :  Olá segue https://www.youtube.com/playlist?list=PLYItvall0TqLBLt6oXFVBaloU7-xZsV-v
  """
  return re.sub('http\S+|www\S+|https\S+', '', texto)

def limpa_pontos(texto) : 
  """
    Função para limpar pontuações 
    Exemplo : eu pago $2.99 por esse aplicativo ... ele deve funcionar ! 
  """
  return re.sub(r'[^\w]', ' ', texto)

def limpa_asc2_emotes(texto) :
    """
    Função para desenhos em asc2 
    Exemplo :      _____
                  /     \
                /- (*) |*)\
                |/\.  _>/\|
                    \__/  
    """
    return re.sub('[\.\,\;\!\?\*\$]','',texto)

def limpeza_simples(texto) : 
  """
    Função que aplica limpa_emotes, limpa_urls, limpa_numeros, limpa_asc2_emotes
    a um comentário
  """
  texto = texto.lower()
  #texto = str( [ word for word in list(texto) if len(word) >= 4] )
  #print(texto)
  texto = limpa_urls(texto)
  texto = limpa_pontos(texto)
  texto = limpa_asc2_emotes(texto)
  texto = limpa_emotes(texto)
  return texto


def preproc_tokenizer(comment, tokenizer,stopwords=stopwords):
    # tokeniza os twtes, usando o tokenizador para essa rede social
    tokens_processados = tokenizer.tokenize(comment) 
    # remove as stopwords
    tokens_processados = [word for word in tokens_processados if word not in stopwords and len(word) > 3]
    # remove pontuação
    tokens_processados = [word for word in tokens_processados if word not in string.punctuation]
    # aplica Lematização
    tokens_processados = [wnl.lemmatize(word) for word in tokens_processados] 

    return tokens_processados


def preproc_entidades_nomeadas(tokens_processados):
    return pos_tag(tokens_processados)

In [7]:
words = []
documents = []
intents = database

classes = [i['tag'] for i in intents['intents']]
ignore_words = STOPWORDS

print(ignore_words)
#intents = json.loads(open('intents.json').read())

# percorremos nosso array de objetos
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)

        # adiciona aos documentos para identificarmos a tag para a mesma
        documents.append((word, intent['tag']))

{"what's", "here's", 'over', "i'll", 'we', 'can', 'it', 'me', 'off', "wasn't", "you'll", 'too', "they'll", "shan't", 'am', 'again', "shouldn't", 'their', 'here', 'http', 'other', "we've", 'when', "we're", "they'd", "couldn't", "how's", 'itself', 'up', 'also', 'but', 'they', 'before', 'yourselves', "you've", 'herself', 'r', 'your', "you're", 'by', "doesn't", "they've", "hasn't", "why's", 'been', 'i', "we'd", 'ourselves', 'a', 'once', 'how', "isn't", 'cannot', 'those', 'yourself', 'ours', 'there', 'until', 'any', 'theirs', "we'll", 'most', "weren't", 'while', 'ought', 'that', "mustn't", "when's", "where's", 'more', "can't", 'however', 'these', 'few', 'after', "didn't", 'otherwise', 'because', "don't", 'who', "i'm", 'whom', 'her', 'his', 'is', 'from', 'hence', 'themselves', 'above', 'further', 'were', 'under', "haven't", "he's", "aren't", 'hers', "he'd", 'com', "won't", 'out', 'if', 'would', 'she', 'having', 'an', 'have', "wouldn't", 'as', 'has', 'in', 'else', 'for', 'them', 'doing', 'get

In [8]:
# lematizamos as palavras ignorando os palavras da lista ignore_words
words =  [ limpeza_simples(w) for w in words ]
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]


# classificamos nossas listas
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# print(words)
# print(classes)
#print(sorted(documents,key=lambda x:x[1]))
# salvamos as palavras e classes nos arquivos pkl
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

display(documents)

[(['Oi'], 'welcome'),
 (['Oi', ',', 'bom', 'dia'], 'welcome'),
 (['Oi', ',', 'boa', 'tarde'], 'welcome'),
 (['bom', 'dia'], 'welcome'),
 (['boa', 'tarde'], 'welcome'),
 (['boa', 'noite'], 'welcome'),
 (['oi', ',', 'boa', 'noite'], 'welcome'),
 (['olá', ',', 'boa', 'noite'], 'welcome'),
 (['oiiiii'], 'welcome'),
 (['Olá'], 'welcome'),
 (['oiii', ',', 'como', 'vai', '?'], 'welcome'),
 (['opa', ',', 'tudo', 'bem', '?'], 'welcome'),
 (['Oi'], 'welcome'),
 (['Oi', ',', 'bom', 'dia'], 'welcome'),
 (['Oi', ',', 'boa', 'tarde'], 'welcome'),
 (['bom', 'dia'], 'welcome'),
 (['boa', 'tarde'], 'welcome'),
 (['boa', 'noite'], 'welcome'),
 (['oi', ',', 'boa', 'noite'], 'welcome'),
 (['olá', ',', 'boa', 'noite'], 'welcome'),
 (['oiiiii'], 'welcome'),
 (['Olá'], 'welcome'),
 (['oiii', ',', 'como', 'vai', '?'], 'welcome'),
 (['opa', ',', 'tudo', 'bem', '?'], 'welcome'),
 (['Oi'], 'welcome'),
 (['Oi', ',', 'bom', 'dia'], 'welcome'),
 (['Oi', ',', 'boa', 'tarde'], 'welcome'),
 (['bom', 'dia'], 'welcome')

In [9]:
corpus = [ " ".join(pattern[0]) for pattern in documents]
print(corpus)

labels = set([ " ".join(pattern[1]) for pattern in documents])
print(labels)

['Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , como vai ?', 'opa , tudo bem ?', 'Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , como vai ?', 'opa , tudo bem ?', 'Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , como vai ?', 'opa , tudo bem ?', 'Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , como vai ?', 'opa , tudo bem ?', 'Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , como vai ?', 'opa , tudo bem ?', 'Oi', 'Oi , bom dia', 'Oi , boa tarde', 'bom dia', 'boa tarde', 'boa noite', 'oi , boa noite', 'olá , boa noite', 'oiiiii', 'Olá', 'oiii , 

In [10]:
# inicializamos o treinamento
training = []
output_empty = [0] * len(classes)
for document in documents:
    # inicializamos o saco de palavras 
    bag = []

    # listamos as palavras do pattern
    pattern_words = document[0]

    # lematizamos cada palavra 
    # na tentativa de representar palavras relacionadas
    pattern_words = [lemmatizer.lemmatize( word.lower()) for word in pattern_words]

    # criamos nosso conjunto de palavras com 1, 
    # se a correspondência de palavras for encontrada no padrão     atual
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)

    # output_row atuará como uma chave para a lista, 
    # onde a saida será 0 para cada tag e 1 para a tag atual
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])
#print(training)
# embaralhamos nosso conjunto de treinamentos e transformamos em numpy array
random.shuffle(training)
training = np.array(training)
# criamos lista de treino sendo x os patterns e y as intenções
x = list(training[:, 0])
y = list(training[:, 1])

print(len(x[0]))

47


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [11]:
x= np.array(x)
y = np.array(y)

In [12]:
# vamos separar a base em conjuntos de treino e de test aleatorioamente
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [13]:
x.shape


(123, 47)

In [14]:
y.shape

(123, 3)

In [15]:
import tensorflow as tf
iShape = (x.shape[1],)
oShape = y.shape[1]

# the deep learning model
ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
ourNewModel.add(Dropout(0.5))
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
md = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=[tf.keras.metrics.Precision()])
print(ourNewModel.summary())
ourNewModel.fit(X_train, y_train, epochs=200, verbose=1 ,  validation_data=(X_test, y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6144      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 14,595
Trainable params: 14,595
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
3/3 [==============================

In [16]:
import random
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))


def clear_writing(writing):
    """
        Limpa todas as sentenças inseridas.
    """

    #tokeniza todas as frases inseridas, lematiza cada uma delas e retorna
    sentence_words = nltk.word_tokenize(writing)
    return [lemmatizer.lemmatize(word.lower()) for word in sentence_words]


# retorna 0 ou 1 para cada palavra da bolsa de palavras


def bag_of_words(writing, words):
    """
        Pega as sentenças que são limpas e cria um pacote de palavras que são usadas 
        para classes de previsão que são baseadas nos resultados que obtivemos treinando o modelo.
    """
    # tokenize the pattern
    sentence_words = clear_writing(writing)

    # cria uma matriz de N palavras
    bag = [0]*len(words)
    for setence in sentence_words:
        for i, word in enumerate(words):
            if word == setence:
                # atribui 1 no pacote de palavra se a palavra atual estiver na posição da frase
                bag[i] = 1

    return(np.array(bag))


def class_prediction(writing, model):
    """
      Faz a previsao do pacote de palavras, usamos como limite de erro 0.25 para evitarmos overfitting
      e classificamos esses resultados por força da probabilidade.
    """

    # filtra as previsões abaixo de um limite 0.25
    prevision = bag_of_words(writing, words)
    response_prediction = model.predict(np.array([prevision]))[0]
    results = [[index, response] for index, response in enumerate(response_prediction) if response > 0.25]    
    #print(results)
    # verifica nas previsões se não há 1 na lista, se não há envia a resposta padrão (anything_else) 
    # ou se não corresponde a margem de erro
    if "1" not in str(prevision) or len(results) == 0 :
        results = [[0, response_prediction[0]]]

    # classifica por força de probabilidade
    results.sort(key=lambda x: x[1], reverse=True)
    print([{"intent": classes[r[0]], "probability": str(r[1])} for r in results])
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]


def get_response(intents, intents_json):
    """
        pega a lista gerada e verifica o arquivo json e produz a maior parte das respostas com a maior probabilidade.
    """
    tag = intents[0]['intent']
    list_of_intents = intents_json['intents']
    for idx in list_of_intents:
        if idx['tag'] == tag:
            # caso as respostas sejam um array contendo mais de uma, 
            # usamos a função de random para pegar uma resposta randomica da nossa lista
            result = random.choice(idx['responses'])
            break

    return result, idx['tag']

In [17]:
# def class_prediction(writting,model):
#     sentence_words = nltk.word_tokenize(writting)
#     write =  [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
#     rep_tf = tfidf.transform(write)
#     response_prediction = model.predict(rep_tf)[0]
#     results = [[index, response] for index, response in enumerate(response_prediction) if response > 0.25] 
#     print(results)
#         # classifica por força de probabilidade
#     results.sort(key=lambda x: x[1], reverse=True)
#     return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

# def get_response(intents, intents_json):
#     """
#         pega a lista gerada e verifica o arquivo json e produz a maior parte das respostas com a maior probabilidade.
#     """
#     tag = intents[0]['intent']
#     list_of_intents = intents_json['intents']
#     for idx in list_of_intents:
#         if idx['tag'] == tag:
#             # caso as respostas sejam um array contendo mais de uma, 
#             # usamos a função de random para pegar uma resposta randomica da nossa lista
#             result = random.choice(idx['responses'])
#             break
#     return result

def conversa(msg):    
    ints = class_prediction(msg, ourNewModel)
    res = get_response(ints, intents)
    return res

In [18]:
# exemplo de caso:
# ola bom dia, meu ra é 11201810691
# se é my_classes procura um ra
# se tiver ra testa se o ra é valido
# se o ra é valido retorna a grade do aluno
# se não é valido ou nao encontrou ra pede o ra
# ----------------------------------------

def valida_ra(message):
  ra = re.findall('\d+', message)

  if ra != []:
    # testa se o user digitou mais de um numero
    if len(ra) > 1:
      return False
    # descarta ra com tamanho diferente de 11 e 9
    elif len(ra[0]) != 11 and len(ra[0]) != 9:
      return False
    else:
      # confirma o ra e retorna a grade para o aluno
      print("Seu ra é esse mesmo? ", ra[0])
      return True

  # retorna false caso nao encontre um ra
  return False


# testa se é my_classes
def ismy_classes(intent):
  if intent == 'my_classes':
    return True  
  return False

In [27]:
def user_input():
  print("mande uma mensagem ao nosso bot, ele reconhece pedidos relacionados as disciplinas e atende saudações!")

  message = "ra é 123456789"
  #message = input()
  res, intent = conversa(message)
  print(res)

  if ismy_classes(intent):
    if valida_ra(message):
      # busca o ra na base de dados
      # se encontrar o ra retorna a grade
      # se nao encontrar pede o ra de novo
      pass
    else:
        print("Digite seu RA, por favor")
        ra = input() 
        while(not valida_ra(ra)):
            print("Digite novamente seu RA, por favor")
            ra = input()
        # faz a query com o ra
  else:
    # se nao tem ra pede para digitar
    pass

user_input()

mande uma mensagem ao nosso bot, ele reconhece pedidos relacionados as disciplinas e atende saudações!
1/1 [==============================] - 0s 14ms/step
[{'intent': 'my_classes', 'probability': '0.9999964'}]
Suas Aulas ?
Seu ra é esse mesmo?  123456789


In [20]:
print("Evaluate on test data")
results = ourNewModel.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

Evaluate on test data
41/41 [==============================] - 0s 1ms/step - loss: 2.9075e-09 - precision: 1.0000
test loss, test acc: [2.907543539265589e-09, 1.0]
